In [7]:
# Cell 1: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import xml.etree.ElementTree as ET
import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import os
import torch.nn as nn

cuda_available = torch.cuda.is_available()

print(f"CUDA Available: {cuda_available}")

if cuda_available:
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. PyTorch will use the CPU.")

CUDA Available: True
GPU 0: NVIDIA GeForce RTX 3080


In [8]:
# Cell 2: Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.images = []
        self.annotations = []
        self.no_annotation_count = 0

        images_dir = os.path.join(root_dir, "raw-images")
        annotations_dir = os.path.join(root_dir, "annotations")

        for subfolder in os.listdir(images_dir):
            subfolder_images_dir = os.path.join(images_dir, subfolder)
            subfolder_annotations_dir = os.path.join(annotations_dir, subfolder)

            if not os.path.isdir(subfolder_annotations_dir):
                continue

            image_files = [f for f in os.listdir(subfolder_images_dir) if f.endswith(".jpg") or f.endswith(".png")]

            image_files.sort()

            num_images = len(image_files)
            if split == "train":
                image_files = image_files[:int(0.7 * num_images)]
            elif split == "val":
                image_files = image_files[int(0.7 * num_images):int(0.9 * num_images)]
            elif split == "test":
                image_files = image_files[int(0.9 * num_images):]
            else:
                raise ValueError(f"Invalid split: {split}")

            for filename in image_files:
                image_path = os.path.join(subfolder_images_dir, filename)
                annotation_path = os.path.join(subfolder_annotations_dir, os.path.splitext(filename)[0] + ".xml")

                if os.path.exists(annotation_path):
                    tree = ET.parse(annotation_path)
                    root = tree.getroot()
                    annotation = []

                    for obj in root.findall("object"):
                        name = obj.find("name").text
                        bbox = obj.find("bndbox")
                        xmin = int(bbox.find("xmin").text)
                        ymin = int(bbox.find("ymin").text)
                        xmax = int(bbox.find("xmax").text)
                        ymax = int(bbox.find("ymax").text)
                        annotation.append((xmin, ymin, xmax, ymax))

                    self.images.append(image_path)
                    self.annotations.append(annotation)
                else:
                    self.no_annotation_count += 1

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = self.images[index]
        image = Image.open(image_path).convert("L")
        annotation = self.annotations[index]

        if self.transform:
            image = self.transform(image)

        if annotation is not None:
            boxes = torch.as_tensor(annotation, dtype=torch.float32)
            labels = torch.ones((len(annotation),), dtype=torch.int64)
        else:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.int64)

        return image, {"boxes": boxes, "labels": labels}

In [9]:
 # Test the dataset
dataset_path = "dataset"  
split = "train"  
thermal_transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

thermal_dataset = CustomDataset(dataset_path, split, thermal_transform)
print(f"Number of images in the dataset: {len(thermal_dataset)}")
image, target = thermal_dataset[0]
print(f"Image shape: {image.shape}")
print(f"Target boxes: {target['boxes']}")
print(f"Target labels: {target['labels']}")

Number of images in the dataset: 857
Image shape: torch.Size([1, 800, 800])
Target boxes: tensor([[367., 137., 407., 153.],
        [238., 125., 279., 145.],
        [ 95., 131., 135., 144.],
        [578., 136., 607., 149.]])
Target labels: tensor([1, 1, 1, 1])


In [10]:
# Cell 3: Dataset Preprocessing
def preprocess_dataset(dataset):
    preprocessed_images = []
    preprocessed_annotations = []
    
    for image, target in dataset:
        if isinstance(image, torch.Tensor):
            image = transforms.ToPILImage()(image)
        else:
            image = Image.fromarray(image)
        
        image = thermal_transform(image)
        
        boxes = target['boxes']
        labels = target['labels']
        
        _, height, width = image.shape
        boxes[:, [0, 2]] /= width
        boxes[:, [1, 3]] /= height
        
        target = {'boxes': boxes, 'labels': labels}
        
        preprocessed_images.append(image)
        preprocessed_annotations.append(target)
    
    return preprocessed_images, preprocessed_annotations

# Test the preprocessing function
dataset_path = "dataset"
split = "train"
thermal_dataset = CustomDataset(dataset_path, split=split, transform=thermal_transform)
preprocessed_images, preprocessed_annotations = preprocess_dataset(thermal_dataset)
print(f"Number of preprocessed images: {len(preprocessed_images)}")
print(f"Number of preprocessed annotations: {len(preprocessed_annotations)}")
print(f"Preprocessed image shape: {preprocessed_images[0].shape}")
print(f"Preprocessed annotation boxes shape: {preprocessed_annotations[0]['boxes'].shape}")
print(f"Preprocessed annotation labels shape: {preprocessed_annotations[0]['labels'].shape}")

Number of preprocessed images: 857
Number of preprocessed annotations: 857
Preprocessed image shape: torch.Size([1, 800, 800])
Preprocessed annotation boxes shape: torch.Size([4, 4])
Preprocessed annotation labels shape: torch.Size([4])


In [11]:
# Cell 4: Dataset and DataLoader Creation
dataset_path = "dataset"
split = "train"

thermal_transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

thermal_dataset = CustomDataset(dataset_path, split, thermal_transform)

class_labels = set()
for _, annotation in thermal_dataset:
    for obj in annotation:
        name = obj[0]
        class_labels.add("vehicle")  

class_to_idx = {"vehicle": 0}  
print("Class labels:", class_to_idx)

preprocessed_thermal_images, preprocessed_thermal_annotations = preprocess_dataset(thermal_dataset)

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    
    images = torch.stack(images, dim=0)
    
    return images, targets

train_thermal_dataset = list(zip(preprocessed_thermal_images, preprocessed_thermal_annotations))
train_thermal_loader = DataLoader(train_thermal_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

for images, targets in train_thermal_loader:
    print(f"Batch images shape: {images.shape}")
    print(f"Batch targets boxes shape: {targets[0]['boxes'].shape}")
    print(f"Batch targets labels shape: {targets[0]['labels'].shape}")
    break

Class labels: {'vehicle': 0}
Batch images shape: torch.Size([4, 1, 800, 800])
Batch targets boxes shape: torch.Size([2, 4])
Batch targets labels shape: torch.Size([2])


In [12]:
# Cell 5: Model Definition and Training
num_classes = len(class_to_idx) + 1

thermal_model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_features = thermal_model.roi_heads.box_predictor.cls_score.in_features
thermal_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
thermal_model.to(device)

thermal_optimizer = torch.optim.SGD(thermal_model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 10

for epoch in range(num_epochs):
    thermal_model.train()
    
    thermal_epoch_loss = 0.0
    
    for thermal_images, thermal_targets in train_thermal_loader:
        thermal_images = list(image.to(device) for image in thermal_images)
        thermal_targets = [{k: v.to(device) for k, v in t.items()} for t in thermal_targets]
        
        thermal_loss_dict = thermal_model(thermal_images, thermal_targets)
        thermal_losses = sum(loss for loss in thermal_loss_dict.values())
        
        thermal_optimizer.zero_grad()
        thermal_losses.backward()
        thermal_optimizer.step()
        
        thermal_epoch_loss += thermal_losses.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Thermal Loss: {thermal_epoch_loss/len(train_thermal_loader):.4f}")

torch.save(thermal_model.state_dict(), "thermal_trained_model.pth")

thermal_model.eval()
with torch.no_grad():
    for images, targets in train_thermal_loader:
        images = list(image.to(device) for image in images)
        outputs = thermal_model(images)
        
        for i in range(len(images)):
            boxes = outputs[i]['boxes'].cpu().numpy()
            labels = outputs[i]['labels'].cpu().numpy()
            scores = outputs[i]['scores'].cpu().numpy()
            
            print(f"Image {i+1} - Boxes: {boxes}, Labels: {labels}, Scores: {scores}")
        
        break

Epoch [1/10], Thermal Loss: 0.3344
Epoch [2/10], Thermal Loss: 0.2084
Epoch [3/10], Thermal Loss: 0.1852
Epoch [4/10], Thermal Loss: 0.1720
Epoch [5/10], Thermal Loss: 0.1494
Epoch [6/10], Thermal Loss: 0.1362
Epoch [7/10], Thermal Loss: 0.1352
Epoch [8/10], Thermal Loss: 0.1345
Epoch [9/10], Thermal Loss: 0.1259
Epoch [10/10], Thermal Loss: 0.1224
Image 1 - Boxes: [[1.4440514e-01 0.0000000e+00 5.1329565e-01 1.8828258e-01]
 [2.9516220e-04 0.0000000e+00 2.4451494e-01 5.3979069e-02]
 [3.6653876e-04 0.0000000e+00 3.0364332e-01 1.6347532e-01]
 [4.1478872e-04 0.0000000e+00 3.4361804e-01 3.8267446e-01]
 [3.1254292e-03 0.0000000e+00 2.5891237e+00 1.1170281e+00]
 [1.5774369e-04 1.7997062e-01 4.0863904e-01 5.0369734e-01]
 [9.4020367e-04 0.0000000e+00 3.0342612e+00 4.6991129e+00]
 [0.0000000e+00 3.6918080e-01 1.6900036e-01 6.5546560e-01]
 [5.3083330e-02 5.8483410e-01 5.7313728e-01 8.9601409e-01]
 [0.0000000e+00 2.7406573e-02 8.3631878e+00 2.0378637e+00]], Labels: [1 1 1 1 1 1 1 1 1 1], Scores: [

In [18]:
# Cell 6: Evaluation and Testing
def evaluate_model(model, dataloader, device):
    model.eval()
    
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for images, targets in dataloader:
            images = list(image.to(device) for image in images)
            outputs = model(images)
            
            for output in outputs:
                boxes = output['boxes'].cpu().numpy()
                labels = output['labels'].cpu().numpy()
                scores = output['scores'].cpu().numpy()
                
                indices = torchvision.ops.nms(torch.tensor(boxes), torch.tensor(scores), iou_threshold=0.5)
                
                filtered_boxes = boxes[indices]
                filtered_labels = labels[indices]
                filtered_scores = scores[indices]
                
                all_predictions.append((filtered_boxes, filtered_labels, filtered_scores))
            
            for target in targets:
                boxes = target['boxes'].cpu().numpy()
                labels = target['labels'].cpu().numpy()
                
                all_targets.append((boxes, labels))
    
    return all_predictions, all_targets

test_split = "test"
test_thermal_dataset = CustomDataset(dataset_path, test_split, thermal_transform)
test_thermal_loader = DataLoader(test_thermal_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

test_predictions, test_targets = evaluate_model(thermal_model, test_thermal_loader, device)
print(f"Number of test predictions: {len(test_predictions)}")
print(f"Number of test targets: {len(test_targets)}")
print(f"Test prediction boxes shape: {test_predictions[0][0].shape}")
print(f"Test prediction labels shape: {test_predictions[0][1].shape}")
print(f"Test prediction scores shape: {test_predictions[0][2].shape}")

Number of test predictions: 96
Number of test targets: 96
Test prediction boxes shape: (3, 4)
Test prediction labels shape: (3,)
Test prediction scores shape: (3,)


In [19]:
# Cell 7: Load the trained model
thermal_model.load_state_dict(torch.load("thermal_trained_model.pth"))
thermal_model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [20]:
# Cell 8: Prepare the test dataset
test_split = "test"
test_thermal_dataset = CustomDataset(dataset_path, test_split, thermal_transform)
test_thermal_loader = DataLoader(test_thermal_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [21]:
# Cell 9: Evaluate the model on the test dataset
test_predictions, test_targets = evaluate_model(thermal_model, test_thermal_loader, device)

unique_labels = np.unique(labels)
print("Unique labels:", unique_labels)
print(f"Boxes: {boxes}")
print(f"Labels: {labels}")
print(f"Scores: {scores}")
print(f"Number of images without annotations: {thermal_dataset.no_annotation_count}")

Unique labels: [1]
Boxes: [[0.         0.         4.0505667  0.93579984]
 [0.         0.         0.36498535 0.21676181]
 [0.         0.48720837 6.433292   3.51608   ]]
Labels: [1 1 1]
Scores: [0.57887954 0.57574505 0.2017587 ]
Number of images without annotations: 556


In [ ]:
# Cell 10: Visualize the object detection results
def visualize_detections(image, boxes, labels, scores, class_labels, confidence_threshold=0.3):
    image_with_detections = image.copy()
    
    height, width, _ = image.shape
    
    if not isinstance(boxes, (list, np.ndarray)):
        boxes = [boxes]
    
    if not isinstance(labels, (list, np.ndarray)):
        labels = [labels]
    
    if not isinstance(scores, (list, np.ndarray)):
        scores = [scores]
    
    for box, label, score in zip(boxes, labels, scores):
        if score >= confidence_threshold:
            if isinstance(box, (list, np.ndarray)):
                xmin, ymin, xmax, ymax = box
            else:
                xmin, ymin, xmax, ymax = box, box, box, box  
            xmin = int(xmin * width)
            ymin = int(ymin * height)
            xmax = int(xmax * width)
            ymax = int(ymax * height)
            
            class_name = class_labels[int(label)]  
            
            cv2.rectangle(image_with_detections, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(image_with_detections, f"{class_name}: {score:.2f}", (xmin, ymin - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return image_with_detections

num_visualizations = 100
class_labels = {1: "vehicle"}

for i in range(num_visualizations):
    image_path = test_thermal_dataset.images[i]
    image = cv2.imread(image_path)  
    
    boxes, labels, scores = test_predictions[i]
    
    print(f"Boxes: {boxes}")
    print(f"Labels: {labels}")
    print(f"Scores: {scores}")
    
    image_with_detections = visualize_detections(image, boxes, labels, scores, class_labels)
    
    cv2.imshow(f"Thermal Object Detection - Image {i+1}", image_with_detections)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Boxes: [[1.8335581e-03 1.0966137e-04 1.8601387e+00 8.1020698e-02]
 [1.0514259e-03 1.2660325e-03 1.0666612e+00 9.3538296e-01]
 [7.1495771e-03 6.4822876e-01 2.6503434e+00 2.4773221e+00]]
Labels: [1 1 1]
Scores: [0.6815838 0.6815838 0.3915118]
Boxes: [[1.0897875e-02 3.3786297e-03 5.2997055e+00 1.2932562e+00]
 [2.9684305e-03 2.4314225e-04 3.0428452e+00 1.3912584e-01]
 [2.1483898e-03 4.6317577e-03 2.2022190e+00 2.6503310e+00]
 [4.3947250e-04 5.2430862e-01 2.3991865e-01 7.3749572e-01]]
Labels: [1 1 1 1]
Scores: [0.6813971 0.6660963 0.6660963 0.5281401]
Boxes: [1.5589137  0.43310583 1.6835008  0.44995666]
Labels: 1
Scores: 0.050927381962537766
Boxes: [0.26329446 0.3962513  0.36859357 0.40955332]
Labels: 1
Scores: 0.6580466628074646
Boxes: [[3.1607151e-03 1.7845631e-03 3.3380859e+00 1.0217369e+00]
 [2.8942823e-03 2.1560943e-01 2.0855079e+00 2.6415477e+00]
 [1.6445667e-04 2.3436764e-01 1.0687785e-01 4.2720738e-01]
 [0.0000000e+00 1.7145276e-04 6.4876194e+00 4.1138060e-02]]
Labels: [1 1 1 1]
Sco